# Keras 学习入门 #

在过年的几天之内，我接触了深度学习框架keras，相对于以前接触的学习框架，theano，lasagne等，这个框架集成度更高，请大家与我一起来学习这篇入门课程，将带领大家一起来学习keras的框架，一定不会令大家失望。

### 首先介绍一下我的机器环境 ###

1. Anaconda 
2. keras 0.3.2
3. jupyter

In [1]:
import keras

In [2]:
keras.__version__

'0.3.2'

#### 这篇教程使用cifra10 作为教程，一步一步实现使用keras实现识别的功能 ####

首先导入必要的库

In [3]:
from keras.datasets import cifar10
from keras.utils import np_utils

导入数据集

In [4]:
nb_classes = 10
batch_size = 32
nb_epoch = 200
img_rows,img_cols = 32,32
img_channels = 3
def load_dataset():
    # 读取数据集
    (X_train,y_train),(X_test,y_test) = cifar10.load_data()
    print 'X_train shape:',X_train.shape
    print X_train.shape[0],'train samples'
    print X_test.shape[0],'test samples'
    
    # 将类别转化为向量
    Y_train = np_utils.to_categorical(y_train,nb_classes)
    Y_test = np_utils.to_categorical(y_test,nb_classes)
    
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /=255
    
    return X_train,Y_train,X_test,Y_test

**下面来构建神经网络**

In [5]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD

Using Theano backend.
Using gpu device 0: GeForce GT 635M (CNMeM is disabled)
D:\Anaconda\lib\site-packages\theano-0.8.0.dev0-py2.7.egg\theano\tensor\signal\downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [6]:
def make_network():
    model = Sequential()
    model.add(Convolution2D(32,3,3,border_mode='same',input_shape=(img_channels,img_rows,img_cols)))
    model.add(Activation('relu'))
    model.add(Convolution2D(32,3,3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Convolution2D(64,3,3,border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64,3,3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    
    return model

开始训练这个模型

In [7]:
def train_model(model,X_train,Y_train,X_test,Y_test):
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd)
    model.fit(X_train, Y_train, nb_epoch=nb_epoch, batch_size=batch_size,
             validation_split=0.1, show_accuracy=True, verbose=1)
    print('Testing...')
    res = model.evaluate(X_test, Y_test,
                        batch_size=batch_size, verbose=1, show_accuracy=True)
    print('Test accuracy: {0}'.format(res[1]))

In [8]:
X_train,Y_train,X_test,Y_test = load_dataset()

X_train shape: (50000L, 3L, 32L, 32L)
50000 train samples
10000 test samples


训练模型

In [10]:
model = make_network()

In [11]:
train_model(model,X_train,Y_train,X_test,Y_test)

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 354s - loss: 1.6543 - acc: 0.3944 - val_loss: 1.2448 - val_acc: 0.5466
Epoch 2/200
45000/45000 [==============================] - 355s - loss: 1.2764 - acc: 0.5420 - val_loss: 1.1139 - val_acc: 0.6176
Epoch 3/200
45000/45000 [==============================] - 358s - loss: 1.1241 - acc: 0.6000 - val_loss: 0.9491 - val_acc: 0.6714
Epoch 4/200
45000/45000 [==============================] - 359s - loss: 1.0254 - acc: 0.6396 - val_loss: 0.8924 - val_acc: 0.6888
Epoch 5/200
45000/45000 [==============================] - 364s - loss: 0.9468 - acc: 0.6660 - val_loss: 0.8758 - val_acc: 0.6990
Epoch 6/200
45000/45000 [==============================] - 357s - loss: 0.8973 - acc: 0.6837 - val_loss: 0.8283 - val_acc: 0.7082
Epoch 7/200
45000/45000 [==============================] - 356s - loss: 0.8591 - acc: 0.7003 - val_loss: 0.7813 - val_acc: 0.7266
Epoch 8/200
45000/45000 [================

## 存储模型 ##

In [13]:
model_json = model.to_json()
open('cifar10_architecture.json', 'w').write(model_json)
model.save_weights('cifar10_weights.h5', overwrite=True)

这样 我们训练好的模型已经可以了，下次不需要再训练，直接拿过来用就OK了

## 下面来使用这个模型来预测图片 ##

### 首先预处理图片 ###

In [14]:
import numpy as np
import scipy.misc

In [35]:
def load_and_scale_imgs():
   img_names = ['standing-cat.jpg', ]
 
   imgs = [np.transpose(scipy.misc.imresize(scipy.misc.imread(img_name), (32, 32)),
                        (2, 0, 1)).astype('float32')
           for img_name in img_names]
   return np.array(imgs) / 255

## 读取模型 ##

In [31]:
from keras.models import model_from_json
 
def load_model(model_def_fname, model_weight_fname):
   model = model_from_json(open(model_def_fname).read())
   model.load_weights(model_weight_fname)
   return model

## 开始预测 ##

In [36]:
imgs = load_and_scale_imgs()

In [37]:
predictions = model.predict_classes(imgs)

1/1 [==============================] - 0s


In [38]:
print(predictions)

[3]


In [39]:
def predictImage(url):
    img_names = [url, ]
    imgs = [np.transpose(scipy.misc.imresize(scipy.misc.imread(img_name), (32, 32)),
                        (2, 0, 1)).astype('float32')
           for img_name in img_names]
    imgs = np.array(imgs) / 255
    predictions = model.predict_classes(imgs)
    print predictions

In [40]:
predictImage('cat1.jpg')

1/1 [==============================] - 0s
[3]


In [41]:
predictImage('images.jpg')

1/1 [==============================] - 0s
[5]


In [42]:
predictImage('index.jpg')

1/1 [==============================] - 0s
[2]


可见，判断率